In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from implementations import *
from IPython.core.display import display, HTML
from cross_validation import *
from build_polynomial import *
%load_ext autoreload
%autoreload 2
from proj1_helpers import *
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))


## Load the training data into feature matrix, class labels, and event ids:

In [2]:
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
#y, tX, ids = load_csv_data(DATA_TRAIN_PATH)
#print(y.shape, tX.shape, ids.shape)

In [3]:
import pandas as pd
df = pd.read_csv("../data/train.csv")
df.tail()

,Id,Prediction,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
249995,349995,b,-999.000,71.989,36.548,5.042,-999.0,-999.0,-999.0,1.392,...,2.859,144.665,0,-999.000,-999.0,-999.000,-999.0,-999.0,-999.0,0.000
249996,349996,b,-999.000,58.179,68.083,22.439,-999.0,-999.0,-999.0,2.585,...,-0.867,80.408,0,-999.000,-999.0,-999.000,-999.0,-999.0,-999.0,0.000
249997,349997,s,105.457,60.526,75.839,39.757,-999.0,-999.0,-999.0,2.390,...,-2.890,198.907,1,41.992,1.8,-0.166,-999.0,-999.0,-999.0,41.992
249998,349998,b,94.951,19.362,68.812,13.504,-999.0,-999.0,-999.0,3.365,...,0.811,112.718,0,-999.000,-999.0,-999.000,-999.0,-999.0,-999.0,0.000
249999,349999,b,-999.000,72.756,70.831,7.479,-999.0,-999.0,-999.0,2.025,...,-1.596,99.405,0,-999.000,-999.0,-999.000,-999.0,-999.0,-999.0,0.000


In [41]:
y0, tX0, id0 = load_data_jet_number(DATA_TRAIN_PATH, 0)
y1, tX1, id1 = load_data_jet_number(DATA_TRAIN_PATH, 1)
y2, tX2, id2 = load_data_jet_number(DATA_TRAIN_PATH, 2)
y3, tX3, id3 = load_data_jet_number(DATA_TRAIN_PATH, 3)
tX0_standardized, tX0_mean, tX0_std= standardize(tX0)
tX1_standardized, tX1_mean, tX1_std= standardize(tX1)
tX2_standardized, tX2_mean, tX2_std= standardize(tX2)
tX3_standardized, tX3_mean, tX3_std= standardize(tX3)

## Do your thing crazy machine learning thing here :) ...

In [42]:
tX0_standardized = np.c_[np.ones((y0.shape[0], 1)), tX0_standardized] #VERY IMPORTANT
y0=y0.reshape(-1,1)
y0=(y0+1)/2
tX1_standardized = np.c_[np.ones((y1.shape[0], 1)), tX1_standardized] #VERY IMPORTANT
y1=y1.reshape(-1,1)
y1=(y1+1)/2
tX2_standardized = np.c_[np.ones((y2.shape[0], 1)), tX2_standardized] #VERY IMPORTANT
y2=y2.reshape(-1,1)
y2=(y2+1)/2
tX3_standardized = np.c_[np.ones((y3.shape[0], 1)), tX3_standardized] #VERY IMPORTANT
y3=y3.reshape(-1,1)
y3=(y3+1)/2

In [47]:
#lambda_=0.0000001
k_fold=4
seed=5
degree=5 #will -1 as range exclude last elem
lower_lambda=-4
upper_lambda=0

weights_0, loss_0, deg0 = cross_validation_best_weight(y0, tX0_standardized, k_fold, degree, seed, lower_lambda, upper_lambda, "jet0")
weights_1, loss_1, deg1 = cross_validation_best_weight(y1, tX1_standardized, k_fold, degree, seed, lower_lambda, upper_lambda, "jet1")
weights_2, loss_2, deg2 = cross_validation_best_weight(y2, tX2_standardized, k_fold, degree, seed, lower_lambda, upper_lambda, "jet2")
weights_3, loss_3, deg3 = cross_validation_best_weight(y3, tX3_standardized, k_fold, degree, seed, lower_lambda, upper_lambda, "jet3")

DEGREE = 1
Lambda = 0.0001
K-Fold =  0
Current iteration = 0, loss = [[52012.95256167]] Current iteration = 500, loss = [[36846.07900451]] rate = [[3.71681075]]
Current iteration = 1000, loss = [[35465.51670272]] rate = [[2.0439779]]
Current iteration = 1500, loss = [[34667.32629415]] rate = [[1.24060774]]
Current iteration = 2000, loss = [[34167.06062111]] rate = [[0.8039582]]
Current iteration = 2500, loss = [[33834.69945044]] rate = [[0.54852636]]


KeyboardInterrupt: 

# Generate predictions and save ouput in csv format for submission:

In [ ]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download test data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
print(tX_test.shape, ids_test.shape)

In [ ]:
_, tX0_te, id0_te = load_data_jet_number("../data/test.csv", 0)
_, tX1_te, id1_te = load_data_jet_number("../data/test.csv", 1)
_, tX2_te, id2_te = load_data_jet_number("../data/test.csv", 2)
_, tX3_te, id3_te = load_data_jet_number("../data/test.csv", 3)
tX0_te_standardized, tX0_te_mean, tX0_te_std= standardize(tX0_te)
tX1_te_standardized, tX1_te_mean, tX1_te_std= standardize(tX1_te)
tX2_te_standardized, tX2_te_mean, tX2_te_std= standardize(tX2_te)
tX3_te_standardized, tX3_te_mean, tX3_te_std= standardize(tX3_te)

In [ ]:
OUTPUT_PATH = '../data/pred.csv' # TODO: fill in desired name of output file for submission
y0_pred = predict_labels(weights_0, build_poly(tX0_te_standardized,deg0))
y1_pred = predict_labels(weights_1, build_poly(tX1_te_standardized,deg1))
y2_pred = predict_labels(weights_2, build_poly(tX2_te_standardized,deg2))
y3_pred = predict_labels(weights_3, build_poly(tX3_te_standardized,deg3))

In [ ]:
y_pred_all=np.concatenate([y0_pred,y1_pred,y2_pred,y3_pred])
ids_pred_all=np.concatenate([id0_te,id1_te,id2_te,id3_te])
y_and_ids=list(zip(*sorted(zip(ids_pred_all, y_pred_all))))
ids_test=y_and_ids[0]
y_pred=y_and_ids[1]

In [ ]:
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)